John Benton / 12-10-2019

# A Comparison of Bostons Neighborhoods:
## Gaining Insights on Major Real Estate Factors 

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)



## Introduction: Business Problem<a name="introduction"></a>

##### Background:

The real estate market has always been a bit frightening for first time home buyers. A home, for a large majority of people, will be the largest purchase of their entire life. For that reason, people rightfully want to get it right. There are a large list of factors that go into the consideration of purchasing a home. In larger cities you can expect some added pressure. Housing has becoming increasingly scarce and people are being forced to make quicker and quicker decisions in order to close the deal before someone steals it out from under them.

##### Problem/ Motivation:

The home buying process is complicated. The information required to make an informed decision on buying a home is vast. That information needs collected and decided upon quickly. Whether you are a person just trying to make an informed and quick decision, you are a person just unfamiliar with the area and need to have a better understanding of all the different neighborhoods, or you are the real estate agent just looking to boost efficiency, I hope to provide some extra information to aid in that process.

We are going to attempt to provide a grouping of neighborhood that have similar characteristics. This will hopefully allow people to look at neighborhoods they might not have considered.

## Data <a name="data"></a>

Since our goal for this project is to group/cluster similar neighborhoods together, I wanted to collect the information that is most important to choosing a neighborhood to live in. For the purpose of this exercise we are going to be looking at crime data that is provided by data.boston.gov, We are going to be looking at high school rankings based on a USNews.com article, and we are going to be collecting a list of venues of things to do in each neighborhood. The list of venues are going to come from foursquare locations api.

The crime data and the high school rankings are going to be based from 2019 data, but have the potential to be expanded upon later. The venue data will be the most up to date data from December of 2019, since we are actually searching for the data around that time.

Boston Crime Data: https://data.boston.gov/dataset/crime-incident-reports-august-2015-to-date-source-new-system/resource/12cb3883-56f5-47de-afa5-3b1cf61b257b

Boston Neighborhood Geo Spatial Data: https://data.boston.gov/dataset/boston-neighborhoods

Boston Highschool Rankings: https://www.usnews.com/education/best-high-schools/massachusetts/districts/boston-public-schools-111992

Venue data: https://foursquare.com/

##### explanation of data considerations:

I briefly wanted to address why these fields were chosen and other categories were left out. I feel like the categories listed above are going to be things that you will definitely experience in the neighborhood. Everyone one is concerned with safety. I feel like most people looking to live in a city would like ample things in the area to do, such as coffee shops or museums. I realize high school rankings might not apply to everyone, but I think a majority of people probably consider the education of the area for potential children.

I chose to leave out factors like commute time, because that can vary so much depending on the person applying. I also left out average real-estate prices because I wanted to try to allow for the clustering algorithm later to not be influenced by over inflated prices of an area. I wanted this data comparison to be strictly on what I would call "the merits" of the neighborhood. Just because an area is expensive doesn't inherently make it more fun, more safe, or better educated. 

I also chose to leave out things like elementary school and middle school rankings for simplicity. I also didn't want to add to many similar categories at the risk of adding noise. I know early education is important, but for the first pass at this data i'm going to only consider high school rankings.